<a href="https://colab.research.google.com/github/A-Peoples/BCS-Recreated/blob/main/BCS_Recreated_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
# installing sportsdataverse
!pip install sportsdataverse
!pip install xgboost==1.6.2
import numpy as np

In [2]:
import sportsdataverse as sdv
import pandas as pd
import numpy as np

In [3]:
data = sdv.cfb.load_cfb_schedule([2025], return_as_pandas=True)
data["margin"] = data["home_points"] - data["away_points"]
team_names = data['home_team'].unique()

data = data.dropna(subset=['home_points', 'away_points'])

data.shape

100%|██████████| 1/1 [00:00<00:00,  2.20it/s]


(3699, 32)

In [69]:
data["margin"] = data["home_points"] - data["away_points"]

final_team = pd.DataFrame()
wins = 0
losses = 0
c_wins = 0
c_losses = 0
margin = 0
total_games = 0
conference = ''
global team_opp_eilo
for team_name in team_names:
  team_opp_eilo = []
  points_scored = 0
  points_against = 0
  eilo = 0
  data["margin"] = data["home_points"] - data["away_points"]
  data
  team = data.loc[(data['home_team'] == team_name) | (data['away_team'] == team_name)]
  team
  main_team = pd.DataFrame()
  for index, row in team.iterrows():
    if row["home_team"] == team_name:
      points_scored = points_scored + row['home_points']
      points_against = points_against + row['away_points']
      total_games +=1
      eilo = eilo + row["home_pregame_elo"]
      team_opp_eilo.append(row["away_pregame_elo"])
      if row["home_points"] >  row["away_points"]:
        if row["conference_game"] == True:
          c_wins +=1
        wins +=1
      elif row["home_points"] <  row["away_points"]:
        if row["conference_game"] == True:
          c_losses +=1
        losses +=1
      margin += row["margin"]
      conference = row['home_conference']
    elif row["away_team"] == team_name:
      points_against = points_against + row['home_points']
      points_scored = points_scored + row['away_points']
      eilo = eilo + row["away_pregame_elo"]
      team_opp_eilo.append(row["home_pregame_elo"])
      total_games +=1
      if row["home_points"] <  row["away_points"]:
        if row["conference_game"] == True:
          c_wins +=1
        wins +=1
      elif row["home_points"] >  row["away_points"]:
        if row["conference_game"] == True:
          c_losses +=1
        losses +=1
      margin += row["margin"]
      conference = row['away_conference']
      points_scored = points_scored / total_games
      points_against = points_against / total_games
  main_team = pd.DataFrame({
      "team": [team_name],
      "wins": [wins],
      "losses": [losses],
      "c_wins": [c_wins * 0.5],
      "c_losses": [c_losses * 1.5],
      'conference': [conference],
      'points_scored': [points_scored],
      'points_against': [points_against],
      "margin": [margin],
      'eilo': [eilo],
      'games_played': [total_games]
      })
  final_team = pd.concat([final_team, main_team], ignore_index=True)
  wins = 0
  losses = 0
  c_wins = 0
  c_losses = 0
  margin = 0
  total_games = 0

In [62]:
FBS = ('SEC', 'Sun Belt', 'Big Ten', 'Pac-12', 'Conference USA', 'FBS Independents', 'ACC', 'Mountain West', 'Big 12', "Mid American")

In [75]:
FBS = final_team.loc[final_team['conference'].isin(['SEC', 'Sun Belt', 'Big Ten', 'Pac-12', 'Conference USA', 'FBS Independents', 'ACC', 'Mountain West', 'Big 12', "Mid American"])]

FBS.eilo = FBS['eilo'] / FBS['games_played']

FBS["points"] = ((FBS['wins'] * 15) - (FBS['losses']) * 25) + ((FBS['c_wins'] * 15) - (FBS['c_losses']) * 15) + (FBS['margin'] / 8) + (FBS['eilo'] / 10)

FBS = FBS.sort_values(by=["points"], ascending=False)
FBS["rank"] = range(1, len(FBS) + 1)
FBS = FBS.head(25)
FBS = FBS[["rank","team","conference", "wins", "losses", "points", 'margin','eilo']]


/tmp/ipython-input-450337305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FBS.eilo = FBS['eilo'] / FBS['games_played']
/tmp/ipython-input-450337305.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FBS["points"] = ((FBS['wins'] * 15) - (FBS['losses']) * 25) + ((FBS['c_wins'] * 15) - (FBS['c_losses']) * 15) + (FBS['margin'] / 8) + (FBS['eilo'] / 10)


**Run This**

In [76]:
FBS = FBS[["rank","team","conference", "wins", "losses", "points", 'margin','eilo']]
FBS

,rank,team,conference,wins,losses,points,margin,eilo
81,1,Ohio State,Big Ten,12,0,469.908333,127,2065.333333
105,2,Indiana,Big Ten,12,0,468.316667,169,1996.916667
119,3,Oregon,Big Ten,11,1,391.375,108,2003.75
110,4,Georgia,SEC,11,1,386.075,99,2037.0
166,5,Ole Miss,SEC,11,1,384.075,154,1948.25
129,6,James Madison,Sun Belt,11,1,377.958333,54,1712.083333
160,7,Texas Tech,Big 12,11,1,374.375,151,1780.0
167,8,BYU,Big 12,11,1,363.5,127,1701.25
147,9,Texas A&M,SEC,11,1,361.425,95,1795.5
405,10,Alabama,SEC,10,2,347.141667,189,1935.166667
